In [ ]:
# Import packages
import os
import time
import tensorflow as tf
from google.colab import drive

# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

# Import Remaining Packages (Post-Install)
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql.functions import col, count


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [864 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,104 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,235 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcont

In [ ]:
# Create a SparkSession
spark = SparkSession.builder.appName("SparkML").getOrCreate()

In [ ]:
#connect to google drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# double check to make sure your pathway to the csv's is the same!!!!

#read in the csv data from google drive
# spark.sparkContext.addFile('/content/gdrive/MyDrive/Bootcamp/Project4/Data/train_timeseries.csv')
# train_timeseries_df = spark.read.csv(SparkFiles.get('/content/gdrive/MyDrive/Bootcamp/Project4/Data/train_timeseries.csv'), sep=",", header=True, inferSchema=True)

spark.sparkContext.addFile('/content/gdrive/MyDrive/Bootcamp/Project4/Data/test_timeseries.csv')
test_timeseries_df = spark.read.csv(SparkFiles.get('/content/gdrive/MyDrive/Bootcamp/Project4/Data/test_timeseries.csv'), sep=",", header=True, inferSchema=True)

# spark.sparkContext.addFile('/content/gdrive/MyDrive/Bootcamp/Project4/Data/validation_timeseries.csv')
# validation_timeseries_df = spark.read.csv(SparkFiles.get('/content/gdrive/MyDrive/Bootcamp/Project4/Data/validation_timeseries.csv'), sep=",", header=True, inferSchema=True)

spark.sparkContext.addFile('/content/gdrive/MyDrive/Bootcamp/Project4/Data/soil_data.csv')
soil_data_df = spark.read.csv(SparkFiles.get('/content/gdrive/MyDrive/Bootcamp/Project4/Data/soil_data.csv'), sep=",", header=True, inferSchema=True)

In [ ]:
# # Read in CSVs into Data Frames
# spark.sparkContext.addFile('soil_data.csv')
# soil_data_df = spark.read.csv(SparkFiles.get('soil_data.csv'), sep=",", header=True, inferSchema=True)

# spark.sparkContext.addFile('test_timeseries.csv')
# test_timeseries_df = spark.read.csv(SparkFiles.get('test_timeseries.csv'), sep=",", header=True, inferSchema=True)

In [ ]:
soil_data_df.show()

+----+---------+----------+---------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------------+-----------------+----------------+-----------------+-----------------+----------------+-----------------+-------------------+-------------------+---+---+---+---+---+---+---+
|fips|      lat|       lon|elevation|slope1|slope2|slope3|slope4|slope5|slope6|slope7|slope8|aspectN|aspectE|aspectS|aspectW|aspectUnknown|         WAT_LAND|        NVG_LAND|         URB_LAND|         GRS_LAND|        FOR_LAND|      CULTRF_LAND|        CULTIR_LAND|          CULT_LAND|SQ1|SQ2|SQ3|SQ4|SQ5|SQ6|SQ7|
+----+---------+----------+---------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------------+-----------------+----------------+-----------------+-----------------+----------------+-----------------+-------------------+-------------------+---+---+---+---+---+---+---+
|1001|32.536382| -86.64449|       63|0.0419|0.2788|0.2984|

In [ ]:
test_timeseries_df.show()

+----+----------+-------+------+----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+
|fips|      date|PRECTOT|    PS|QV2M|  T2M|T2MDEW|T2MWET|T2M_MAX|T2M_MIN|T2M_RANGE|   TS|WS10M|WS10M_MAX|WS10M_MIN|WS10M_RANGE|WS50M|WS50M_MAX|WS50M_MIN|WS50M_RANGE|score|
+----+----------+-------+------+----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+
|1001|2019-01-01|   2.25|100.51|9.69|14.71| 13.55| 13.52|  17.38|  10.92|     6.46|14.63|  1.2|      1.5|     0.79|       0.71| 2.74|     4.01|     1.23|       2.78|  0.0|
|1001|2019-01-02|   4.94|100.48|8.65|13.05| 11.83| 11.74|  17.76|   9.54|     8.23|13.06| 1.02|     1.35|     0.32|       1.03| 2.13|     3.37|     0.56|       2.81| null|
|1001|2019-01-03|  20.74|100.03|8.59|12.12| 11.67| 11.67|  13.74|  10.44|      3.3|12.12| 1.83|     4.23|     0.34|       3.88| 3.41|     7.

In [ ]:
# filled_test_timeseries_df = test_timeseries_df.fillna({'score':5})
#filled_test_timeseries_df = test_timeseries_df

from pyspark.sql import functions as F


modified_test_timeseries_df = test_timeseries_df.withColumn('score', F.col('score') + 1)
filled_test_timeseries_df = modified_test_timeseries_df.fillna({'score':0}) # Fill null values with 0


In [ ]:
joined_df = filled_test_timeseries_df.join(soil_data_df, on='fips', how='left')
# Round Score column
from pyspark.sql.functions import round, col
from pyspark.sql.types import IntegerType

# Round the values in the column
# joined_df = joined_df.withColumn("score", round(col("score")))

# Cast the rounded values to integer
#joined_df = joined_df.withColumn("score", col("score").cast("integer"))

joined_df.show()

+----+----------+-------+------+----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+---------+---------+---------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------------+-----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+---+---+---+---+---+---+---+
|fips|      date|PRECTOT|    PS|QV2M|  T2M|T2MDEW|T2MWET|T2M_MAX|T2M_MIN|T2M_RANGE|   TS|WS10M|WS10M_MAX|WS10M_MIN|WS10M_RANGE|WS50M|WS50M_MAX|WS50M_MIN|WS50M_RANGE|score|      lat|      lon|elevation|slope1|slope2|slope3|slope4|slope5|slope6|slope7|slope8|aspectN|aspectE|aspectS|aspectW|aspectUnknown|         WAT_LAND|        NVG_LAND|        URB_LAND|        GRS_LAND|        FOR_LAND|     CULTRF_LAND|     CULTIR_LAND|       CULT_LAND|SQ1|SQ2|SQ3|SQ4|SQ5|SQ6|SQ7|
+----+----------+-------+------+----+-----+------+------+-----

In [ ]:
#joined_df = joined_df.withColumn("score", round(joined_df["score"]).cast(IntegerType()))

In [ ]:
# Count of Unique Locations in Soil Data
soil_locations = soil_data_df.groupBy('fips').agg(count("*").alias("count"))
soil_locations.count()
print(f"Soil Data contains {soil_locations.count()} unique locations")

# Count of Unique Locations in Test Timeseries
test_timeseries_locations = test_timeseries_df.groupBy('fips').agg(count("*").alias("count"))
test_timeseries_locations.count()
print(f"Test Timeseries contains {test_timeseries_locations.count()} unique locations")
print("---")

# Print shapes of dataframes
print(f"Soil Data contains {soil_data_df.count()} rows and {len(soil_data_df.columns)} columns")
print(f"Test Timeseries contains {test_timeseries_df.count()} rows and {len(test_timeseries_df.columns)} columns")
print(f"Joined contains {joined_df.count()} rows and {len(joined_df.columns)} columns")

Soil Data contains 3109 unique locations
Test Timeseries contains 3108 unique locations
---
Soil Data contains 3109 rows and 32 columns
Test Timeseries contains 2271948 rows and 21 columns
Joined contains 2271948 rows and 52 columns


In [ ]:
joined_df.printSchema()

root
 |-- fips: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- PRECTOT: double (nullable = true)
 |-- PS: double (nullable = true)
 |-- QV2M: double (nullable = true)
 |-- T2M: double (nullable = true)
 |-- T2MDEW: double (nullable = true)
 |-- T2MWET: double (nullable = true)
 |-- T2M_MAX: double (nullable = true)
 |-- T2M_MIN: double (nullable = true)
 |-- T2M_RANGE: double (nullable = true)
 |-- TS: double (nullable = true)
 |-- WS10M: double (nullable = true)
 |-- WS10M_MAX: double (nullable = true)
 |-- WS10M_MIN: double (nullable = true)
 |-- WS10M_RANGE: double (nullable = true)
 |-- WS50M: double (nullable = true)
 |-- WS50M_MAX: double (nullable = true)
 |-- WS50M_MIN: double (nullable = true)
 |-- WS50M_RANGE: double (nullable = true)
 |-- score: double (nullable = false)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- elevation: integer (nullable = true)
 |-- slope1: double (nullable = true)
 |-- slope2: double (nullable = true)
 

In [ ]:
from pyspark.sql import functions as F
joined_df_scores = joined_df.groupBy('score').agg(count("*").alias("count")).orderBy(F.desc("count"))
joined_df_scores.show()

+------+-------+
| score|  count|
+------+-------+
|   0.0|1945608|
|   1.0| 222634|
|   2.0|  12196|
|   3.0|   5585|
|   4.0|   1875|
|   5.0|    781|
|   6.0|    225|
|1.0001|    150|
|1.0003|    119|
|1.0005|     91|
|1.0004|     89|
|1.0002|     87|
|1.0012|     65|
|1.0008|     64|
| 1.001|     61|
|1.0015|     57|
|1.0013|     56|
|1.0007|     53|
|1.9998|     50|
|1.0011|     47|
+------+-------+
only showing top 20 rows



In [ ]:
summary_df = joined_df.summary("min", "max")

# Print the results
summary_df.show()

+-------+-----+-------+------+-----+------+------+------+-------+-------+---------+------+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+---------+-----------+---------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------------+--------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+---+---+---+---+---+---+---+
|summary| fips|PRECTOT|    PS| QV2M|   T2M|T2MDEW|T2MWET|T2M_MAX|T2M_MIN|T2M_RANGE|    TS|WS10M|WS10M_MAX|WS10M_MIN|WS10M_RANGE|WS50M|WS50M_MAX|WS50M_MIN|WS50M_RANGE|score|      lat|        lon|elevation|slope1|slope2|slope3|slope4|slope5|slope6|slope7|slope8|aspectN|aspectE|aspectS|aspectW|aspectUnknown|WAT_LAND|        NVG_LAND|        URB_LAND|        GRS_LAND|        FOR_LAND|     CULTRF_LAND|     CULTIR_LAND|       CULT_LAND|SQ1|SQ2|SQ3|SQ4|SQ5|SQ6|SQ7|
+-------+-----+-------+------+-----+------+------+------+-------+-------+-

In [ ]:
# Columns that we do not want to include as features
exclude_columns = ['score', 'lat', 'lon', 'fips', 'date']

# Set feature columns (excluding non-feature columns)
feature_columns = [col_name for col_name in joined_df.columns if col_name not in exclude_columns]
feature_columns

['PRECTOT',
 'PS',
 'QV2M',
 'T2M',
 'T2MDEW',
 'T2MWET',
 'T2M_MAX',
 'T2M_MIN',
 'T2M_RANGE',
 'TS',
 'WS10M',
 'WS10M_MAX',
 'WS10M_MIN',
 'WS10M_RANGE',
 'WS50M',
 'WS50M_MAX',
 'WS50M_MIN',
 'WS50M_RANGE',
 'elevation',
 'slope1',
 'slope2',
 'slope3',
 'slope4',
 'slope5',
 'slope6',
 'slope7',
 'slope8',
 'aspectN',
 'aspectE',
 'aspectS',
 'aspectW',
 'aspectUnknown',
 'WAT_LAND',
 'NVG_LAND',
 'URB_LAND',
 'GRS_LAND',
 'FOR_LAND',
 'CULTRF_LAND',
 'CULTIR_LAND',
 'CULT_LAND',
 'SQ1',
 'SQ2',
 'SQ3',
 'SQ4',
 'SQ5',
 'SQ6',
 'SQ7']

In [ ]:
# Create the VectorAssembler instance to combine feature columns into a single dense vector column
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
assembler

VectorAssembler_8ce7c769d2af

In [ ]:
# Apply the VectorAssembler to create the dense vector column
df_with_features = assembler.transform(joined_df)
df_with_features.show()

+----+----------+-------+------+----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+---------+---------+---------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------------+-----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+---+---+---+---+---+---+---+--------------------+
|fips|      date|PRECTOT|    PS|QV2M|  T2M|T2MDEW|T2MWET|T2M_MAX|T2M_MIN|T2M_RANGE|   TS|WS10M|WS10M_MAX|WS10M_MIN|WS10M_RANGE|WS50M|WS50M_MAX|WS50M_MIN|WS50M_RANGE|score|      lat|      lon|elevation|slope1|slope2|slope3|slope4|slope5|slope6|slope7|slope8|aspectN|aspectE|aspectS|aspectW|aspectUnknown|         WAT_LAND|        NVG_LAND|        URB_LAND|        GRS_LAND|        FOR_LAND|     CULTRF_LAND|     CULTIR_LAND|       CULT_LAND|SQ1|SQ2|SQ3|SQ4|SQ5|SQ6|SQ7|            features|
+----+----------+---

In [ ]:
# Apply feature scaling using StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)
scaler

StandardScaler_b2a742a9a653

In [ ]:
# Fit the StandardScaler to compute mean and standard deviation
scaler_model = scaler.fit(df_with_features)
scaler_model

StandardScalerModel: uid=StandardScaler_b2a742a9a653, numFeatures=47, withMean=true, withStd=true

In [ ]:
# Transform the DataFrame to get scaled feature vectors
df_with_scaled_features = scaler_model.transform(df_with_features)
df_with_scaled_features.show()

+----+----------+-------+------+----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+---------+---------+---------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------------+-----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+---+---+---+---+---+---+---+--------------------+--------------------+
|fips|      date|PRECTOT|    PS|QV2M|  T2M|T2MDEW|T2MWET|T2M_MAX|T2M_MIN|T2M_RANGE|   TS|WS10M|WS10M_MAX|WS10M_MIN|WS10M_RANGE|WS50M|WS50M_MAX|WS50M_MIN|WS50M_RANGE|score|      lat|      lon|elevation|slope1|slope2|slope3|slope4|slope5|slope6|slope7|slope8|aspectN|aspectE|aspectS|aspectW|aspectUnknown|         WAT_LAND|        NVG_LAND|        URB_LAND|        GRS_LAND|        FOR_LAND|     CULTRF_LAND|     CULTIR_LAND|       CULT_LAND|SQ1|SQ2|SQ3|SQ4|SQ5|SQ6|SQ7|            features|

In [ ]:
df_with_scaled_features.show()

+----+----------+-------+------+----+-----+------+------+-------+-------+---------+-----+-----+---------+---------+-----------+-----+---------+---------+-----------+-----+---------+---------+---------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------------+-----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+---+---+---+---+---+---+---+--------------------+--------------------+
|fips|      date|PRECTOT|    PS|QV2M|  T2M|T2MDEW|T2MWET|T2M_MAX|T2M_MIN|T2M_RANGE|   TS|WS10M|WS10M_MAX|WS10M_MIN|WS10M_RANGE|WS50M|WS50M_MAX|WS50M_MIN|WS50M_RANGE|score|      lat|      lon|elevation|slope1|slope2|slope3|slope4|slope5|slope6|slope7|slope8|aspectN|aspectE|aspectS|aspectW|aspectUnknown|         WAT_LAND|        NVG_LAND|        URB_LAND|        GRS_LAND|        FOR_LAND|     CULTRF_LAND|     CULTIR_LAND|       CULT_LAND|SQ1|SQ2|SQ3|SQ4|SQ5|SQ6|SQ7|            features|

In [ ]:
# Split the preprocessed data into a training and testing dataset
train_data, test_data = df_with_scaled_features.randomSplit([0.8, 0.2], seed=42)

In [ ]:
#unique_values_list = df_with_scaled_features.select('score').distinct().rdd.flatMap(lambda x: x).collect()
#unique_values_list[0]+1

In [ ]:
#df_with_scaled_features = df_with_scaled_features.withColumnRenamed("score", "score1")

In [ ]:
#df_with_scaled_features = df_with_scaled_features.withColumn("label", df["label"].cast(IntegerType()))

In [ ]:
from pyspark.ml.regression import LinearRegression

# Define the model
lr = LinearRegression(featuresCol='scaled_features', labelCol='score')

# Train the model
lr_model = lr.fit(train_data)

# Make predictions
predictions = lr_model.transform(test_data)

# Evaluate the model
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="score", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = " + str(rmse))

Root Mean Squared Error (RMSE) on test data = 0.5846329237269471


In [ ]:
summary_df = joined_df.summary()
summary_df.show()